# Домашнее задание
## Harry Potter and the Action Prediction Challenge from Natural Language

*deadline*: 14 ноября 2022, 23:59

В этом домашнем задании вы будете работать с корпусом Harry Potter and the Action Prediction Challenge. Корпус собран из фанфиков о Гарри Поттере и состоит из двух частей: 1) сырые тексты, 2) фрагменты текстов, описывающих ситуацию, в которой произнесено заклинание.

Корпус описан в статье: https://arxiv.org/pdf/1905.11037.pdf

David Vilares and Carlos Gómez-Rodríguez. Harry Potter and the Action Prediction Challenge from Natural Language. 2019 Annual Conference of the North American Chapter of the Association for Computational Linguistics. To appear.

Код для сбора корпуса находится в репозитории: https://github.com/aghie/hpac . Корпус можно скачать по инструкции из этого репозитория, но для экономии времени авторы задания уже скачали и подготовили данные к работе. 

Ссылки на собранный корпус: 
* Сырые тексты:  https://www.dropbox.com/s/23xet9kvbqna1qs/hpac_raw.zip?dl=0
* Токенизированные тексты в нижнем регистре: https://www.dropbox.com/s/gwfgmomdbetvdye/hpac_lower_tokenized.zip?dl=0
* train-test-dev: https://www.dropbox.com/s/3vdz0mouvex8abd/hpac_splits.zip?dl=0

Части 1, 2 задания должны быть выполнены на полных текстах (сырых или предобработанных -- на ваше усмотрение), Часть 3 – на разбиение на тестовое, отладочное и обучающее множества. Тестовое множество должно быть использовано исключительно для тестирования моделей, обучающее и отладочное – для выбора модели и параметров. 

В статье и репозитории вы найдете идеи, которые помогут вам выполнить домашнее задание. Их стоит воспринимать как руководство к действию, и не стоит их копировать и переиспользовать. Обученные модели использовать не нужно, код для их обучения можно использовать как подсказку. 

## ПРАВИЛА
1. Домашнее задание выполняется в группе до 3-х человек.
2. Домашнее задание оформляется в виде отчета либо в .pdf файле, либо ipython-тетрадке. 
3. Отчет должен содержать: нумерацию заданий и пунктов, которые вы выполнили, код решения, и понятное пошаговое описание того, что вы сделали. Отчет должен быть написан в академическом стиле, без излишнего использования сленга и с соблюдением норм русского языка.
4. Не стоит копировать фрагменты лекций, статей и Википедии в ваш отчет.
5. Отчеты, состоящие исключительно из кода, не будут проверены и будут автоматически оценены нулевой оценкой.
6. Плагиат и любое недобросоветсное цитирование приводит к обнуление оценки. 


## Часть 1. [2 балла] Эксплоративный анализ 
1. Найдите топ-1000 слов по частоте без учета стоп-слов.
2. Найдите топ-10 по частоте: имен, пар имя + фамилия, пар вида ''профессор'' + имя / фамилия. 

[бонус] Постройте тематическую модель по корпусу HPAC.

[бонус] Найдите еще что-то интересное в корпусе (что-то специфичное для фанфиков или фентези-тематики)



### Данные
Токенизированные тексты 

In [ ]:
!unzip hpac_source

Посмотрим на предложенные предобработанные тексты:

In [3]:
! cat hpac_source/3777622 | wc -w

3207


В целом, в предобработанной версии не хватает только лемматизации.

In [6]:
# Imports 

import os
import pandas as pd
import nltk
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
from nltk.corpus import stopwords
from tqdm import tqdm
import hw.tg_utils

In [7]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
sent_detector.PUNCTUATION


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/paperspace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/paperspace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/paperspace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


(';', ':', ',', '.', '!', '?')

In [8]:
path = "."

corpdir = path + '/hpac_source/' # Tokenized
files = os.listdir(corpdir)

hpac_corpus = PlaintextCorpusReader(corpdir, files, sent_tokenizer=sent_detector)

In [9]:
from string import punctuation

sw = set(stopwords.words("english") + list(sent_detector.PUNCTUATION) + list(punctuation) + ["``", "--", "n", "''", "..."])

In [13]:
fd_words = nltk.FreqDist()

files  = hpac_corpus.fileids()

for f in tqdm(files, desc="Files"):
    f_words = []
    for word in hpac_corpus.words(f):
        if word not in sw:
            f_words.append(word)
    fd_words.update(f_words)

Files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36225/36225 [13:20<00:00, 45.25it/s]


In [14]:
import pandas as pd

df_w = pd.DataFrame(fd_words.most_common(), columns=["word", "count"])
df_w.to_csv("words_common.csv", index=None)

In [16]:
fd_bigrams = nltk.FreqDist()

files  = hpac_corpus.fileids()

for f in tqdm(files, desc="Files"):
    f_words = []
    for word in hpac_corpus.words(f):
        if word not in sw:
            f_words.append(word)
    bg = nltk.bigrams(f_words)
    fd_bigrams.update(bg)


Files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36225/36225 [16:49<00:00, 35.87it/s]


In [17]:
fd_bigrams.most_common()

[(('harry', 'said'), 175945),
 (('dark', 'lord'), 142495),
 (('shook', 'head'), 139433),
 (('harry', 'potter'), 135085),
 (('death', 'eaters'), 125541),
 (('could', 'see'), 99501),
 (('hermione', 'said'), 84896),
 (('let', 'go'), 81562),
 (('death', 'eater'), 80996),
 (('harry', 'asked'), 80622),
 (('common', 'room'), 74405),
 (('said', 'harry'), 73734),
 (('first', 'time'), 70701),
 (('harry', 'could'), 69758),
 (('great', 'hall'), 69655),
 (('harry', 'looked'), 62890),
 (('ron', 'hermione'), 61470),
 (('could', 'help'), 60503),
 (('even', 'though'), 57769),
 (('would', 'never'), 57288),
 (('would', 'like'), 57064),
 (('rolled', 'eyes'), 56423),
 (('make', 'sure'), 55201),
 (('draco', 'said'), 53683),
 (('looked', 'like'), 53647),
 (('made', 'way'), 52549),
 (('harry', 'ron'), 52268),
 (('draco', 'malfoy'), 50640),
 (('could', 'feel'), 49780),
 (('looked', 'around'), 49512),
 (('ron', 'said'), 48291),
 (('felt', 'like'), 47831),
 (('harry', 'hermione'), 46477),
 (('professor', 'snape'

In [19]:
df_bigram = pd.DataFrame(fd_bigrams.most_common(), columns=["bigram", "freq"])

In [ ]:
df_bigram.to_csv("bigram_common.csv", index=None)

In [23]:
fd_bigrams.most_common(100000)

[(('harry', 'said'), 175945),
 (('dark', 'lord'), 142495),
 (('shook', 'head'), 139433),
 (('harry', 'potter'), 135085),
 (('death', 'eaters'), 125541),
 (('could', 'see'), 99501),
 (('hermione', 'said'), 84896),
 (('let', 'go'), 81562),
 (('death', 'eater'), 80996),
 (('harry', 'asked'), 80622),
 (('common', 'room'), 74405),
 (('said', 'harry'), 73734),
 (('first', 'time'), 70701),
 (('harry', 'could'), 69758),
 (('great', 'hall'), 69655),
 (('harry', 'looked'), 62890),
 (('ron', 'hermione'), 61470),
 (('could', 'help'), 60503),
 (('even', 'though'), 57769),
 (('would', 'never'), 57288),
 (('would', 'like'), 57064),
 (('rolled', 'eyes'), 56423),
 (('make', 'sure'), 55201),
 (('draco', 'said'), 53683),
 (('looked', 'like'), 53647),
 (('made', 'way'), 52549),
 (('harry', 'ron'), 52268),
 (('draco', 'malfoy'), 50640),
 (('could', 'feel'), 49780),
 (('looked', 'around'), 49512),
 (('ron', 'said'), 48291),
 (('felt', 'like'), 47831),
 (('harry', 'hermione'), 46477),
 (('professor', 'snape'

In [8]:
!ls hpac_source | wc -l

36225


In [ ]:
!unzip hpac_splits

## Часть 2. [2 балла] Модели представления слов 
Обучите модель представления слов (word2vec, GloVe, fastText или любую другую) на материале корпуса HPAC.
1. Продемонстрируйте, как работает поиск синонимов, ассоциаций, лишних слов в обученной модели. 
2. Визуализируйте топ-1000 слов по частоте без учета стоп-слов (п. 1.1) с помощью TSNE или UMAP (https://umap-learn.readthedocs.io).

In [108]:
! pip install gensim

Defaulting to user installation because normal site-packages is not writeable


In [109]:
import gensim

In [ ]:
gensim.models.FastText.load_fasttext_format()

In [50]:
def corpus_generator(sw):
    for f in hpac_corpus.fileids():
        for sentence in hpac_corpus.sents(f):
            f_words = []
            for word in sentence:
                if word not in sw:
                    f_words.append(word)
            yield f_words

In [28]:
from gensim import corpora

In [47]:
model = Word2Vec(workers=32, min_count=5, window=5, sample=1e-3)

In [48]:
fd_words = nltk.FreqDist()

files  = hpac_corpus.fileids()
sw = set(list(sent_detector.PUNCTUATION) + list(punctuation) + ["``", "--", "n", "''", "..."])

for f in tqdm(files, desc="Files"):
    f_words = []
    for word in hpac_corpus.words(f):
        if word not in sw:
            f_words.append(word)
    fd_words.update(f_words)

Files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 36225/36225 [15:45<00:00, 38.29it/s]


In [49]:
model.build_vocab_from_freq(fd_words)

In [53]:
model.train(corpus_generator(sw=sw), total_examples=model.corpus_count, epochs=1, )

Exception in thread Thread-38:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/paperspace/.local/lib/python3.8/site-packages/gensim/models/word2vec.py", line 1219, in _job_producer
    epoch_progress = 1.0 * pushed_words / total_words
TypeError: unsupported operand type(s) for /: 'float' and 'NoneType'


KeyboardInterrupt: 

In [33]:
# w2vcorp = corpora.Dictionary(corpus_generator())

KeyboardInterrupt: 

In [ ]:
model = Word2Vec(workers=32, size=100, min_count=5, window=5, sample=1e-3)
model.build_vocab(corpus_generator())
model.train(corpus_generator(), total_examples=model.corpus_count, epochs=10, )

## Часть 3. [5 баллов] Классификация текстов
Задача классификации формулируется так: данный фрагмент фанфика описывают какую-то ситуацию, которая предшествует произнесению заклинания. Требуется по тексту предсказать, какое именно заклинание будет произнесено. Таким образом, заклинание - это фактически метка класса. Основная мера качества – macro $F_1$.
Обучите несколько классификаторов и сравните их между собой. Оцените качество классификаторов на частых и редких классах. Какие классы чаще всего оказываются перепутаны? Связаны ли ошибки со смыслом заклинаний?

Используйте фрагменты из множества train для обучения, из множества dev для отладки, из множества test – для тестирования и получения итоговых результатов. 

1. [1 балл] Используйте fastText в качестве baseline-классификатора.
2. [2 балла] Используйте сверточные  или реккурентные сети в качестве более продвинутого классификатора. Поэкспериментируйте с количеством и размерностью фильтров, используйте разные размеры окон, попробуйте использовать $k$-max pooling. 
3. [2 балла] Попробуйте расширить обучающее множество за счет аугментации данных. Если вам понадобится словарь синонимов, можно использовать WordNet (ниже вы найдете примеры).

[бонус] Используйте результат max pooling'а как эмбеддинг входного текста. Визуализируйте эмбеддинги 500-1000 предложений из обучающего множества и изучите свойства получившегося пространства.

[бонус] Используйте ваш любимый классификатор и любые (честные) способы повышения качества классификации и получите macro $F_1$ больше 0.5.

## Часть 4. [1 балл] Итоги
Напишите краткое резюме проделанной работы. Читали ли вы сами Гарри Поттера или фанфики о нем и помогло ли вам знание предметной области в выполнении домашнего задания?

In [122]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from sklearn.metrics import f1_score
from torchtext.vocab import Vocab
from torchtext.vocab import vocab as torchVocab

In [123]:
import pandas as pd

train_df = pd.read_csv("./hpac_corpus/hpac_training_128.tsv", sep="\t", header=None, names=["score", "spell", "sent"])
dev_df = pd.read_csv("./hpac_corpus/hpac_dev_128.tsv", sep="\t", header=None, names=["score", "spell", "sent"])
test_df = pd.read_csv("./hpac_corpus/hpac_test_128.tsv", sep="\t", header=None, names=["score", "spell", "sent"])

In [124]:
from torchtext.transforms import RegexTokenizer
from nltk.tokenize import RegexpTokenizer
from string import punctuation

def preprocess_data(df):
    punct = set(list(punctuation) + ["``", "--", "n", "''", "..."])

    tokenizer = RegexpTokenizer(r"\w+")
    def preprocess_sent(s):
        tokens = tokenizer.tokenize(s)
        res = []
        for tok in tokens:
            if tok not in punct:
                res.append(tok)
        return " ".join(res)
    df["preprocess_sent"] = df["sent"].apply(preprocess_sent)
    return df

In [125]:
train_df = preprocess_data(train_df)
test_df = preprocess_data(test_df)
dev_df = preprocess_data(dev_df)

In [126]:
w2v = gensim.models.fasttext.load_facebook_vectors("/home/paperspace/hw-1-nlp-hse-2022-takkat14/hw/word_reprs_fasttext.bin")

In [127]:
labels = set(train_df["spell"].values).union(set(test_df["spell"].values)).union(set(dev_df["spell"].values))
label2idx = {label:i for i, label in enumerate(labels)}
idx2label = {label2idx[label]:label for label in labels}

In [128]:
import numpy as np

In [245]:
w2v.most_similar(["harry"])

[('hermione', 0.8506463766098022),
 ('draco', 0.8191326856613159),
 ('ron', 0.8067686557769775),
 ('ginny', 0.8017417192459106),
 ('hermionie', 0.7206811904907227),
 ('hermoine', 0.7177939414978027),
 ('katherine', 0.7153907418251038),
 ('severus', 0.7114371657371521),
 ('hermoini', 0.7104672193527222),
 ('jamie', 0.7067298889160156)]

In [261]:
from torch.utils.data import IterableDataset


class HPAC_Dataset(IterableDataset):
    def __init__(self, df, label_col="spell", sent_col="preprocess_sent", vector_model=w2v, aug=False, aug_prob=0.15, emb_dim=100, max_len=256, label2idx=label2idx):
        self.data = df[[label_col, sent_col]]
        self.model = vector_model
        self.aug = aug
        self.max_len = max_len
        self.emb_dim = emb_dim 
        self.label2idx = label2idx     
        self.aug_prob = aug_prob  
        
        
    def __iter__(self):
        for i, sample in self.data.iterrows():
            features = np.zeros((self.max_len, self.emb_dim))
            for j, word in enumerate(sample[1].split(" ")):
                if j == self.max_len:
                    break
                if not self.aug:
                    features[j] = self.model.get_vector(word)
                else:
                    if i % 100 == 0 and np.random.rand() > 1 - self.aug_prob:
                        choose = np.random.choice(10)
                        if np.random.rand() > 0.5:
                            features[j] = self.model.get_vector(self.model.most_similar(positive=[word])[choose][0])
                        else:
                            features[j] = self.model.get_vector(self.model.most_similar(negative=[word])[choose][0])
            
            yield torch.tensor(features.astype(np.float32)), label2idx[sample[0]]


In [188]:
train_dataset = HPAC_Dataset(train_df)
val_dataset = HPAC_Dataset(dev_df)

In [199]:
class CNNModel(nn.Module):
    def __init__(self, kernel_size=3, padding=1, stride=1):
        super().__init__()

        self.model = nn.Sequential(
            nn.Conv1d(256, 100, kernel_size=kernel_size, padding=padding),
            nn.ReLU(),
            nn.Conv1d(100, 16, kernel_size=kernel_size, padding=kernel_size),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(1664, 256),
            nn.ReLU(),
            nn.Linear(256, 85)
            )


    def forward(self, x):
        return self.model(x)


In [211]:
import torch

model = CNNModel()
optimizer = optim.Adam(model.parameters())
device = 'cpu'  if torch.cuda.is_available() else torch.cuda.current_device()

model.to(device)
    

CNNModel(
  (model): Sequential(
    (0): Conv1d(256, 100, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(100, 16, kernel_size=(3,), stride=(1,), padding=(3,))
    (3): ReLU()
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=1664, out_features=256, bias=True)
    (6): ReLU()
    (7): Linear(in_features=256, out_features=85, bias=True)
  )
)

In [212]:
BATCH_SIZE = 64

train_loader = DataLoader(train_dataset, pin_memory=True, batch_size=BATCH_SIZE)
val_loader = DataLoader(val_dataset, shuffle=False, pin_memory=True, batch_size=BATCH_SIZE)
    

In [266]:
def train_eval_loop(model, opt, tdl, vdl, num_epochs=5, device=device):
    mean_train_accs = []
    mean_val_accs = []
    mean_train_losses = []
    mean_val_losses = []
    for epoch in tqdm(range(num_epochs), desc="Epocs"):
        model.train()
        accs = []
        val_accs = []
        losses = []
        val_losses = []
        for sample, label in tqdm(tdl, desc="Batch", leave=False):
            opt.step()
            sample = sample.to(device)
            logits = model(sample)

            loss = F.cross_entropy(logits, label)
            losses.append(loss.item())
            acc = (logits.argmax(axis=1) == label).float().mean().cpu().numpy()
            accs.append(acc)

            opt.zero_grad()
            loss.backward()
        
        print(f"Train accuracy: {np.mean(accs)}")
        print(f"Train loss: {np.mean(losses)}")
        mean_train_accs.append(np.mean(accs))
        mean_train_losses.append(np.mean(losses))
        
        model.eval()
        
        for sample, label in tqdm(vdl, desc="Batch", leave=False):
            sample = sample.to(device)
            logits = model(sample)
            loss = F.cross_entropy(logits, label)
            val_losses.append(loss.item())
            acc = (logits.argmax(axis=1) == label).float().mean().cpu().numpy()
            val_accs.append(acc)

        print(f"Val loss: {np.mean(val_losses)}")
        print(f"Val accuracy {np.mean(val_accs)}")
        mean_val_accs.append(np.mean(val_accs))
        mean_val_losses.append(np.mean(val_losses))
    return mean_train_accs, mean_val_accs, mean_train_losses, mean_val_losses


            

In [214]:
ta, va, tl, vl = train_eval_loop(model, optimizer, train_loader, val_loader)

Batch: 953it [02:24,  6.59it/s]                                                                                                                                  | 0/5 [00:00<?, ?it/s]


Train accuracy: 0.18932496011257172
Train loss: 3.2116410547138385


Epocs:  20%|████████████████████████████                                                                                                                | 1/5 [02:38<10:34, 158.56s/it]

Val loss: 3.04071447428535
Val accuracy 0.23417681455612183


Batch: 953it [02:21,  6.72it/s]


...

Val loss: 2.912301748740573
Val accuracy 0.2560547888278961


Batch: 953it [02:05,  7.57it/s]


Train accuracy: 0.30991581082344055
Train loss: 2.6478068893628754


Epocs: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [12:50<00:00, 154.14s/it]

Val loss: 2.9306278108548716
Val accuracy 0.2566072344779968


In [218]:
test_dataset = HPAC_Dataset(test_df)


In [221]:
def get_predictions(test_dataset):
    preds = []
    gt = []
    for s, l in test_dataset:
        s.to(device)
        pred = model(s.unsqueeze(0)).argmax().item()
        preds.append(pred)
        gt.append(l)
    return gt, preds


In [229]:
from sklearn.metrics import f1_score, accuracy_score

gt, preds = get_predictions(test_dataset=test_dataset)
print("NO AUG")
print(f"F1 MICRO: {f1_score(gt, preds, average='micro')}")
print(f"F1 MACRO: {f1_score(gt, preds, average='macro')}")
print(f"ACC: {accuracy_score(gt, preds)}")



NO AUG
F1 MICRO: 0.26370621174632114
F1 MACRO: 0.06273495812129097
ACC: 0.26370621174632114


In [234]:
model_f = model

## Augmetations

In [267]:
train_dataset = HPAC_Dataset(train_df, aug=True)
val_dataset = HPAC_Dataset(dev_df, aug=True)

In [268]:
import torch

model = CNNModel()
optimizer = optim.Adam(model.parameters())
device = 'cpu'  if torch.cuda.is_available() else torch.cuda.current_device()

model.to(device)
    

CNNModel(
  (model): Sequential(
    (0): Conv1d(256, 100, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(100, 16, kernel_size=(3,), stride=(1,), padding=(3,))
    (3): ReLU()
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=1664, out_features=256, bias=True)
    (6): ReLU()
    (7): Linear(in_features=256, out_features=85, bias=True)
  )
)

In [269]:
BATCH_SIZE = 64

train_loader = DataLoader(train_dataset, pin_memory=True, batch_size=BATCH_SIZE)
val_loader = DataLoader(val_dataset, shuffle=False, pin_memory=True, batch_size=BATCH_SIZE)
    

In [270]:
ta, va, tl, vl = train_eval_loop(model, optimizer, train_loader, val_loader)

Epocs:   0%|                                                                                                                                                     | 0/5 [00:00<?, ?it/s]/home/paperspace/.local/lib/python3.8/site-packages/gensim/models/fasttext.py:1128: RuntimeWarning: invalid value encountered in divide
  return word_vec / np.linalg.norm(word_vec)


Train accuracy: 0.1235634982585907
Train loss: 3.469348444143097


Epocs:  20%|████████████████████████████                                                                                                                | 1/5 [03:37<14:29, 217.42s/it]

Val loss: 3.4691068885707055
Val accuracy 0.12195774912834167


Epocs:  20%|████████████████████████████                                                                                                                | 1/5 [04:56<19:44, 296.23s/it]


KeyboardInterrupt: 

train, test, dev файлы

In [ ]:
import pandas as pd
df = pd.read_csv('hpac_splits/hpac_training_128.tsv', sep = '\t', header = None)

In [ ]:
df.head()

In [ ]:
df.iloc[0][1], df.iloc[0][2]

### Как использовать WordNet из nltk?

In [ ]:
# скачиваем WordNet
import nltk
nltk.download('wordnet')

In [ ]:
# слово -> множество синсетов (синонимов разных смыслов исходного слова)
from nltk.corpus import wordnet as wn
wn.synsets('magic')

In [ ]:
# посмотрим, что внутри одного синсета
wn.synsets('magic')[1].lemmas()[0]

In [ ]:
# возьмем лемму одного из слов из синсета
wn.synsets('magic')[1].lemmas()[-1].name()